# Naver Movie 평점 데이터 수집
## 알라딘 Aladdin, 2019

In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

In [2]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=163788'
url = url_base + url_sub

html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

### iframe에서 src 태그의 주소를 얻어야 함

In [3]:
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=163788&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

### review 건수 구하기

In [4]:
url = url_base + ifr_url
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")
count = soup.find('strong', 'total')
count

<strong class="total">관람객 평점 <em>26,440</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [5]:
count = count.find('em').string
count 

'26,440'

In [6]:
total_count = int(count.replace(',', ''))
total_count

26440

In [7]:
import math
total_page = math.ceil(total_count / 10)
total_page

2644

### 첫번째 페이지에서 리뷰, 평점, 날짜 구해보기

In [8]:
point = []
review = []
date = []
results = soup.find('div', 'score_result')
lis = results.find_all('li')
for li in lis:
    p = li.find('div', 'star_score').find('em').string
    point.append(p)
    r = li.find('div', 'score_reple').find('p').find('span').string.strip(' \t\n\r')
    review.append(r)
    d = li.find('div', 'score_reple').find_all('em')[1].string
    date.append(d)

In [9]:
import pandas as pd
df = pd.DataFrame({'point':point, 'review':review, 'date':date})
df

,point,review,date
0,10,윌 스미스가 하드캐리하여 의도치 않게 원작을 뛰어넘은 실사영화,2019.05.23 11:26
1,10,영화가 너무 아름답다..ost도 잊혀지지 안고 디즈니 실사중에서 최고중에 최고다 쟈...,2019.05.23 09:11
2,10,존잼임... 난 처음 도입부에 아~라~비안 나~잇 이 부분에서부터 감동받아서 울 뻔...,2019.05.23 21:57
3,10,관람객,2019.05.23 12:55
4,10,A whole new world 나올때때 개소름...,2019.05.23 09:06
5,10,자스민 존나 이쁘다..,2019.05.23 17:36
6,10,지니가 나오면.. 이세상 흥이 아니다ㅎ,2019.05.23 21:54
7,10,일요일 아침 8시 디즈니 만화동산 챙겨보고 비디오 알라딘 애니도 수십번 빌려봐서 테...,2019.05.23 13:06
8,10,디즈니가 미쳤음.. 영화시장을 통재로 잡아먹으려고 작정을함..끝내주는 영화 이제껏 ...,2019.05.23 22:06
9,10,제일 우려스러웠던 윌스미스가 하드캐리하는바람에 원작 뛰어넘어버림 ㅋㅋㅋ,2019.05.24 00:49


### 모든 데이터 리스트에 담기
#### 리뷰 데이터는 여러가지 경우가 있음
- 1. 일반적인 리뷰
- 2. 관람객이라 표시된 경우
- 3. 데이터가 긴 경우
- 4. SPO 인 경우
- 5. 관람객이면서 데이터가 긴 경우

In [10]:
point = []
review = []
date = []
for page in range(1, total_page+1):
    url = url_base + ifr_url + '&page=' + str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    if page % 100 == 0:
        print(page, end=' ')
    results = soup.find('div', 'score_result')
    lis = results.find_all('li')
    for li in lis:
        p = li.find('div', 'star_score').find('em').string
        point.append(p)
        s = li.find('div', 'score_reple').find('p').find_all('span')
        if len(s) == 1:         # case 1
            r = li.find('div', 'score_reple').find('span').string.strip(' \t\n\r')
        elif len(s) == 2:
            s = li.find('div', 'score_reple').find('p').find('span').string
            if s == '관람객':   # case 2
                r = li.find('div', 'score_reple').find_all('span')[1].string.strip(' \t\n\r')
            else:
                s = li.find('div', 'score_reple').find('p').find_all('span')[1].string
                if s is None:   # case 3
                    r = li.find('div', 'score_reple').find('p').find_all('span')[1].find('a')['data-src']
                else:           # case 4
                    r = li.find('div', 'score_reple').find('p').find_all('span')[1].string.strip(' \t\n\r')
        else:                   # case 5
            r = li.find('div', 'score_reple').find('p').find_all('span')[2].find('a')['data-src']
        review.append(r)
        d = li.find('div', 'score_reple').find_all('em')[1].string
        date.append(d)    

100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 

### 데이터 프레임으로 만들기

In [11]:
df = pd.DataFrame({'point':point, 'review':review, 'date':date})
df.shape

(26440, 3)

In [12]:
df.tail()

,point,review,date
26435,3,지니 억텐. 알라딘 찐따 존재감 없음. 노래 시끄러움. 스토리 지겨움. 자스민 미드...,2019.05.29 11:09
26436,1,노래는 뮤지컬영화니까 나오는거고;;그냥 노래를 못 부름,2019.07.09 19:42
26437,1,"엄복동같은 수작은 17만을 보고, 이런 허술한 판타지극은 개봉 7일만에 100만 돌...",2019.06.01 02:26
26438,1,"최악!!!! 내용은 그냥 딱 만화 동화 수준,, 윌스미스 깨방정 조낸 보기 싫고 영...",2019.07.09 21:22
26439,1,알라딘이 결국 죽다니..,2019.05.23 10:36


### 엑셀 파일로 저장

In [13]:
df.to_excel("data/04.NaverMovie_Aladdin.xlsx")